In [2]:
import json
import numpy as np
import torch
import soundfile as sf
import librosa

from pathlib import Path

from nemo.collections.tts.torch.helpers import BetaBinomialInterpolator

In [16]:
generate_mels = "PREDICTED" # or "PREDICTED", "ORIGINAL"
# folder_name = "synmels_r1_999_difflen" # or "orig_synmels", "synmels_r3_149_difflen"
# fastpitch_model_ckpt_path = "/github/FastPitch--v_loss=0.6877-epoch=999-last.ckpt"
folder_name = "synmels_r3_149_difflen"
fastpitch_model_ckpt_path = "/github/FastPitch--v_loss=0.6679-epoch=149.ckpt"
dataset_part = "test" # or "val", "train", "test"
dataset_base_path = "/github/datasets/openslr-95-german-neutral-tts/"

In [4]:
if generate_mels == "PREDICTED":
    from nemo.collections.tts.models import FastPitchModel
    spec_model = FastPitchModel.load_from_checkpoint(fastpitch_model_ckpt_path)
    spec_model.eval().cuda()

[NeMo I 2022-05-12 01:40:12 tokenize_and_classify:93] Creating ClassifyFst grammars.


[NeMo W 2022-05-12 01:40:43 g2ps:84] apply_to_oov_word=None, it means that some of words will remain unchanged if they are not handled by one of rule in self.parse_one_word(). It is useful when you use tokenizer with set of phonemes and chars together, otherwise it can be not.
[NeMo W 2022-05-12 01:40:43 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /Data/openslr-95-german-neutral-tts/thorsten-de/train_manifest.json
      sample_rate: 22050
      sup_data_path: /Data/openslr-95-german-neutral-tts/thorsten-de/sup_data
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null

[NeMo I 2022-05-12 01:40:43 features:259] PADDING: 1
[NeMo I 2022-05-12 01:40:43 features:276] STFT using torch


In [5]:
def infer(spec_gen_model, vocoder_model, str_input, speaker=None):
    """
    Synthesizes spectrogram and audio from a text string given a spectrogram synthesis and vocoder model.
    
    Args:
        spec_gen_model: Spectrogram generator model (FastPitch in our case)
        vocoder_model: Vocoder model (HiFiGAN in our case)
        str_input: Text input for the synthesis
        speaker: Speaker ID
    
    Returns:
        spectrogram and waveform of the synthesized audio.
    """
    with torch.no_grad():
        parsed = spec_gen_model.parse(str_input)
        if speaker is not None:
            speaker = torch.tensor([speaker]).long().to(device=spec_gen_model.device)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed, speaker=speaker)
        audio = vocoder_model.convert_spectrogram_to_audio(spec=spectrogram)
        
    if spectrogram is not None:
        if isinstance(spectrogram, torch.Tensor):
            spectrogram = spectrogram.to('cpu').numpy()
        if len(spectrogram.shape) == 3:
            spectrogram = spectrogram[0]
    if isinstance(audio, torch.Tensor):
        audio = audio.to('cpu').numpy()
    return spectrogram, audio

def get_best_ckpt_from_last_run(
        base_dir, 
        new_speaker_id, 
        duration_mins, 
        mixing_enabled, 
        original_speaker_id, 
        model_name="FastPitch"
    ):    
    mixing = "no_mixing" if not mixing_enabled else "mixing"
    
    d = f"{original_speaker_id}_to_{new_speaker_id}_{mixing}_{duration_mins}_mins"
    
    exp_dirs = list([i for i in (Path(base_dir) / d / model_name).iterdir() if i.is_dir()])
    last_exp_dir = sorted(exp_dirs)[-1]
    
    last_checkpoint_dir = last_exp_dir / "checkpoints"
    
    last_ckpt = list(last_checkpoint_dir.glob('*-last.ckpt'))

    if len(last_ckpt) == 0:
        raise ValueError(f"There is no last checkpoint in {last_checkpoint_dir}.")
    
    return str(last_ckpt[0])

In [17]:
def load_wav(audio_file):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
    return samples.transpose()

In [18]:
# Get records from the training manifest
manifest_path = dataset_base_path+"thorsten-de/"+dataset_part+"_manifest.json"
records = []
with open(manifest_path, "r") as f:
    for i, line in enumerate(f):
        records.append(json.loads(line))

In [19]:
beta_binomial_interpolator = BetaBinomialInterpolator()

if generate_mels == "PREDICTED":
    spec_model.eval()
    device = spec_model.device

save_dir = Path(dataset_base_path+folder_name+"/"+dataset_part)

save_dir.mkdir(exist_ok=True, parents=True)

# Generate a spectrograms (we need to use ground truth alignment for correct matching between audio and mels)
for i, r in enumerate(records):
    
    if generate_mels == "PREDICTED":
        audio = load_wav(r["audio_filepath"])

        audio = torch.from_numpy(audio).unsqueeze(0).to(device)
        audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)

        # Again, our finetuned FastPitch model doesn't use multiple speakers,
        # but we keep the code to support it here for reference
        if spec_model.fastpitch.speaker_emb is not None and "speaker" in r:
            speaker = torch.tensor([r['speaker']]).to(device)
        else:
            speaker = None

        with torch.no_grad():
            if "normalized_text" in r:
                text = spec_model.parse(r["normalized_text"], normalize=False)
            else:
                text = spec_model.parse(r['text'])

            text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)

            spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)

            # Generate attention prior and spectrogram inputs for HiFi-GAN
            attn_prior = torch.from_numpy(
              beta_binomial_interpolator(spect_len.item(), text_len.item())
            ).unsqueeze(0).to(text.device)

#             spectrogram = spec_model.forward(
#               text=text, 
#               input_lens=text_len, 
#               spec=spect, 
#               mel_lens=spect_len, 
#               attn_prior=attn_prior,
#               speaker=speaker,
#             )[0]
            spectrogram = spec_model.generate_spectrogram(
              tokens=text, 
              speaker=speaker,
            )
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram[0].to('cpu').numpy())
        r["mel_filepath"] = str(save_path)
    else:
        # original melspecs
        print("loading original melspecs")
        y, sr = librosa.load(r["audio_filepath"])
        spectrogram2 = np.log(librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, win_length=1024, hop_length=256))
        spectrogram = spectrogram2[ :80, :]
        save_path = save_dir / f"mel_{i}.npy"
        np.save(save_path, spectrogram)
        r["mel_filepath"] = str(save_path)

hifigan_manifest_path = dataset_base_path+folder_name+"/hifigan_"+dataset_part+"_ft.json"

with open(hifigan_manifest_path, "w") as f:
    for r in records:
        f.write(json.dumps(r) + '\n')
# Please do the same for the validation json. Code is omitted.

/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_0.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_1.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_2.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_3.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4.npy


[NeMo W 2022-05-12 01:52:12 tts_tokenizers:351] Text: [dieser PIH1LZ ist zwar nicht giftig, EY1BER0 ungenießBAA1R.] contains unknown char/phoneme: [ß]. Original text: [Dieser Pilz ist zwar nicht giftig, aber ungenießbar.]. Symbol will be skipped.


/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_5.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_6.npy


[NeMo W 2022-05-12 01:52:13 tts_tokenizers:351] Text: [SEY1N fleißigen wichtel machten sich sofort daran.] contains unknown char/phoneme: [ß]. Original text: [Seine fleißigen Wichtel machten sich sofort daran.]. Symbol will be skipped.


/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_7.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_8.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_9.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_10.npy


...

/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4485.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4486.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4487.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4488.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4489.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4490.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4491.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4492.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4493.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4494.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4495.npy
/github/datasets/openslr-95-germ

[NeMo W 2022-05-12 01:54:03 tts_tokenizers:351] Text: [EH1S ist VAO1N vorteil, wenn MAE1N beidfußig schießEH1N KAE1N.] contains unknown char/phoneme: [ß]. Original text: [Es ist von Vorteil, wenn man beidfußig schießen kann.]. Symbol will be skipped.
[NeMo W 2022-05-12 01:54:03 tts_tokenizers:351] Text: [EH1S ist VAO1N vorteil, wenn MAE1N beidfußig schießEH1N KAE1N.] contains unknown char/phoneme: [ß]. Original text: [Es ist von Vorteil, wenn man beidfußig schießen kann.]. Symbol will be skipped.


/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4506.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4507.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4508.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4509.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4510.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4511.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4512.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4513.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4514.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4515.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4516.npy


[NeMo W 2022-05-12 01:54:03 tts_tokenizers:351] Text: [wir bauen auf, wir reißEH1N NIY1DER0 – SOW1 haben wir arbeit immer WIY1DER0.] contains unknown char/phoneme: [ß]. Original text: [Wir bauen auf, wir reißen nieder – so haben wir Arbeit immer wieder.]. Symbol will be skipped.
[NeMo W 2022-05-12 01:54:03 tts_tokenizers:351] Text: [wir bauen auf, wir reißEH1N NIY1DER0 – SOW1 haben wir arbeit immer WIY1DER0.] contains unknown char/phoneme: [–]. Original text: [Wir bauen auf, wir reißen nieder – so haben wir Arbeit immer wieder.]. Symbol will be skipped.
[NeMo W 2022-05-12 01:54:03 tts_tokenizers:351] Text: [auf dem ruckweg durchfließTIY1 das blut DAY1 venen.] contains unknown char/phoneme: [ß]. Original text: [Auf dem Ruckweg durchfließt das Blut die Venen.]. Symbol will be skipped.


/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4517.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4518.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4519.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4520.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4521.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4522.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4523.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4524.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4525.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4526.npy
/github/datasets/openslr-95-german-neutral-tts/synmels_r3_149_difflen/test/mel_4527.npy
/github/datasets/openslr-95-germ